In [3]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [4]:
import numpy as np
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    y = np.zeros_like(vec)
    y[0]=np.linalg.norm(vec)
    u = (vec - y)/np.linalg.norm(vec-y)
    H = np.eye(vec.shape[0]) - 2*np.outer(u, u)

    return H@vec, H


Test your function using tests below:

In [5]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)

In [6]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [7]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    Q = np.eye(m) # Основа для Q из QR-разложения
    
    for i in range(min(m-1, n)):
        vec, H_i = householder(a1[i:, i]) # Применяем хаусхолдер для зануления i-ого столбца под главной диагональю (включительно)
        E = np.eye(i)
        O = np.zeros((i, m-i))
        O1 = np.zeros((m-i, i))
        Q_shtr_i =np.asarray(np.bmat([[E, O], [O1, H_i]])) # Внутренняя функция собирает нужную матрицу их блоков (единичной матрицы, маленькой матрицы из написанной ранее функции-хаусхолдера и блоков нулей), а внешняя переделывает полученный объект в np.массив
        a1 = Q_shtr_i @ a1 # Зануляем часть очередного столбца
        Q = Q @ Q_shtr_i.T # Добавляем очередную итерационную матрицу к конечной Q
        print()             
    return Q, a1
    
# Проверим на простой матрице 

x = np.array([[1,1, 1], [9, 2, 1], [1, 3, 1]])
print(x)
print("Q = ", qr_decomp(x)[0])
print("R = ", qr_decomp(x)[1])
print("QR = ", qr_decomp(x)[0] @ qr_decomp(x)[1])



[[1 1 1]
 [9 2 1]
 [1 3 1]]


Q =  [[ 0.10976426  0.25714373  0.96011922]
 [ 0.98787834 -0.13489507 -0.07680954]
 [ 0.10976426  0.95691192 -0.26883338]]


R =  [[ 9.11043358e+00  2.41481372e+00  1.20740686e+00]
 [ 2.83285228e-17  2.85808934e+00  1.07916058e+00]
 [-5.51776577e-16  2.20404591e-16  6.14476298e-01]]




QR =  [[1. 1. 1.]
 [9. 2. 1.]
 [1. 3. 1.]]


In [8]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [10]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [11]:
from scipy.linalg import qr
qq, rr = qr(a)

print("Q = ", qr_decomp(a)[0])
print("R = ", qr_decomp(a)[1])
print("----------------------------------------------------------------------", "\n")
print(qq, "\n")
print(rr)
assert_allclose(np.dot(qq, rr), a)




Q =  [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]



R =  [[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.         -0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.         -0.         -0.        ]]
---------------------------------------------------------------------- 

[[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 

[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)

Матрицы отличаются, так как QR-разложение не единственно. То есть в библиотечной реализации матрица R выбирается иначе. Можно привести пример: формально( так как определитель таки равен нулю) единичная матрица I может быть представлена в виде QR-разложения как произведение единичных матриц, так единичная матрица одновременно является и верхнетреугольной и ортогональной. Но также разложением будет являться и произведение двух "минус-единичных" матриц. Вот и неоднозначность...
Более общее утверждение состоит в следующем: A = QR <==> A=QPP^(-1)R это верно для некоторых P (для ортогональных и верхнетруегольных одновременно) --> разложение на (QP) и (P^(-1)R) тоже является искомым разложенинем. Но такие Р могут быть только диагональными, и при этом PP^(-1) = I, ---> см.частный случай.(Если матрица не квадаратная, а прямоугольная, то (так как нижние строки (если m>n, то m-n) состоят из нулей)  несколько столбцов может отличаться не только знаком, но и модулем.


# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [12]:
def constr_R(a): # Эта функция собирает матрицу R, а также собирает все векторы u в матрицу U (по строкам)
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    U = np.zeros_like(a1)
    
    for i in range(min(m-1, n)):
        R = a1[i:, i:]
        vec = R[:, 0]
        
        y = np.zeros_like(vec)
        y[0]=np.linalg.norm(vec)
        u = (vec - y)/np.linalg.norm(vec-y) # Применяем почти хаусхолдер для зануления i-ого столбца под главной диагональю (включительно)
        
        R = R - 2*np.outer(u, u.T @ R) # Зануляем часть очередного столбца
        a1[i:, i:] = R # Меняем на "исправленную часть" рассматриваемую часть матрицы а
        U[:len(vec),i]= u # Номером вектора "u" является номер столбца матрицы U, да не очень эстетично, но как смогла.
        
    return(a1, U)

#print(constr_R(x)[0])
                         # ---Можно сравнить полученную матрицу с матрицей-результатом работы библиотечной функции.
print(qr(x)[1], "\n")


def constr_R_Q_U(a, U): # Эта функция принимает на вход начальную матрицу а и массив векторов u(столбики), возвращает R
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    for i in range(min(m-1, n)): # Построим матрицу аналогично
        R = a1[i:, i:]
       
        u = U[:R.shape[0],i]
        R = R - 2*np.outer(u, (u.T) @ R) # Фактически процедура из предыдущей функции, просто векторы не нужно искать
        a1[i:, i:] = R # Меняем на "исправленную часть" рассматриваемую часть матрицы а
    return a1
print(constr_R_Q_U(x, constr_R(x)[1]))

# Выведены матрицы R из разложения двумя способами (моим и библиотечным)


[[-9.11043358 -2.41481372 -1.20740686]
 [ 0.          2.85808934  1.07916058]
 [ 0.          0.         -0.6144763 ]] 

[[9.11043358 2.41481372 1.20740686]
 [0.         2.85808934 1.07916058]
 [0.         0.         0.6144763 ]]


In [13]:
# Проведем  тест на произвольных матрицах
rndm = np.random.RandomState(1234)
z = rndm.uniform(size=(5, 3))

# Сначала проверим матрицу R

bibl_r =  qr(z)[1]
Q_m =  qr_decomp(z)[0]

print("Выводятся две матрицы из R разложения. Первая - полученная двумя моими функциями, вторая - результат разложения с помощью библиотечной функции")
print(constr_R_Q_U(z, constr_R(z)[1]), "\n")
print(bibl_r,"\n" )

print("Так как отличиет только в знаке, то мы победили")






Выводятся две матрицы из R разложения. Первая - полученная двумя моими функциями, вторая - результат разложения с помощью библиотечной функции
[[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.          0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.         -0.        ]] 

[[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]] 

Так как отличиет только в знаке, то мы победили
